In [1]:
import math
from pathlib import Path

import arcpy
from ba_tools import data
import pandas as pd
import numpy as np

from itertools import chain
import json
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

In [2]:
project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [3]:
# pth_bp = Path(r'D:\arcgis\ba_data\Data\Demographic Data\block_data.gdb\USA_ESRI_2019_blocks')
pth_bp = gdb_int/'block_points'
pth_bp

WindowsPath('D:/projects/h3-py/data/interim/interim.gdb/block_points')

In [4]:
df_bp = GeoAccessor.from_featureclass(str(pth_bp))
df_bp = df_bp[['FIPS', 'SHAPE']].copy()
df_bp.head()

,FIPS,SHAPE
0,060971543023019,"{'x': -123.0533251, 'y': 38.34898290000007, 's..."
1,060971543024023,"{'x': -123.04938169999997, 'y': 38.33418040000..."
2,060971543043065,"{'x': -123.08428889999999, 'y': 38.38898740000..."
3,060971543043066,"{'x': -123.09656769999998, 'y': 38.40211740000..."
4,060971543043069,"{'x': -123.09134789999996, 'y': 38.40123120000..."


In [5]:
df_bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463145 entries, 0 to 463144
Data columns (total 2 columns):
FIPS     463145 non-null object
SHAPE    463145 non-null geometry
dtypes: geometry(1), object(1)
memory usage: 7.1+ MB


In [6]:
fnl_df = h3_arcgis.get_nonoverlapping_h3_hexbins_for_points(df_bp, h3_min=6, h3_max=10, min_count=5)
fnl_df.head()

,h3_id,SHAPE,count,h3_lvl
0,862802aefffffff,"{'rings': [[[-124.18050779199996, 40.755939914...",1083,6.0
1,8629a565fffffff,"{'rings': [[[-118.40272631899995, 33.875980660...",1018,6.0
2,8628308f7ffffff,"{'rings': [[[-122.29284349699998, 37.872667939...",1017,6.0
3,862834297ffffff,"{'rings': [[[-122.24458080399995, 37.461168360...",990,6.0
4,8629a1d87ffffff,"{'rings': [[[-117.92105146099999, 34.077663478...",800,6.0


In [ ]:
fnl_df.spatial.plot()

In [9]:
fnl_df.spatial.to_featureclass(gdb_int/'h3_bg_ca_05d')

'D:\\projects\\h3-py\\data\\interim\\interim.gdb\\h3_bg_ca_05d'

In [7]:
fnl_df.to_csv(data_int/'h3_bp_05_09.csv')